# Predict the primary type of crime

Useful links
* https://towardsdatascience.com/pyspark-demand-forecasting-data-science-project-dae14b5319cc
* https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SQLContext
sc = pyspark.SparkContext(master='spark://192.168.11.239:7077', appName='type_predicter')
sqlContext = SQLContext(sc)

In [2]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.sql.types import StringType
from datetime import datetime
import pyspark.sql.functions as F #avoid conflicts with regular python functions
from pyspark.sql.functions import udf

## Load the dataset and look at the variables

In [3]:
df = sqlContext.read.csv("/datasets/district11.csv", header='true')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



## Preprocessing
Figure out which features we will be able to provide during a live prediction. We will probably only have time and an uncertain location description to base our prediction on.

First step is to divide the date column into smaller subsets of time. Which one we should keep will be decided later when we tune the parameters.

In [4]:
df = sqlContext.read.csv("/datasets/district11.csv", header='true')

In [5]:
df = (df
       .withColumn('Timestamps', F.to_timestamp("Date", 'MM/dd/yyyy hh:mm:ss a'))
       .withColumn('Day', F.to_date("Date", 'MM/dd/yyyy hh:mm:ss a'))
       .withColumn("Month", F.month("Day"))
       .withColumn("Hour", F.hour("Timestamps"))
       .withColumn("Minute", F.minute("Timestamps"))
       .withColumn("DayOfMonth", F.dayofmonth("Day"))
       .withColumn("DayOfYear", F.dayofyear("Day"))
       .withColumn("DayOfWeek", F.dayofweek("Day"))
       .withColumn('WeekOfYear', F.weekofyear("Day"))
       .withColumn('Quarter', F.quarter("Timestamps"))
       
      )

In [6]:
df.select("Day","Year","Month","DayOfMonth","DayOfYear","DayOfWeek","Hour","Minute","WeekOfYear","Quarter").show(5)

+----------+----+-----+----------+---------+---------+----+------+----------+-------+
|       Day|Year|Month|DayOfMonth|DayOfYear|DayOfWeek|Hour|Minute|WeekOfYear|Quarter|
+----------+----+-----+----------+---------+---------+----+------+----------+-------+
|2007-01-01|2007|    1|         1|        1|        2|   0|     1|         1|      1|
|2008-01-01|2008|    1|         1|        1|        3|   0|     1|         1|      1|
|2017-09-01|2017|    9|         1|      244|        6|   9|     0|        35|      3|
|2018-02-04|2018|    2|         4|       35|        1|  15|    25|         5|      1|
|2014-11-01|2014|   11|         1|      305|        7|   9|     0|        44|      4|
+----------+----+-----+----------+---------+---------+----+------+----------+-------+
only showing top 5 rows



Exact location description such as longitude and latitude will be hard to provide. However, we should be able to provide down to Beat areas etc. Also, in the best case we might have an intuition of the location description. 

Next step will be to drop unrelevant columns

In [7]:
cols = ["Day","Year","Month","Hour","Minute","DayOfMonth","DayOfYear","DayOfWeek","WeekOfYear","Quarter",
       "Beat","Location Description","Primary Type"]
df = df.select(*cols)

Now, lets see if some columns contains N/A values. Ignore columns with date type since they will fail in this case, and the date derivatives would have failed earlier on.
Inspired by https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe

In [8]:
df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) \
           for (c,col_type) in df.dtypes if col_type not in ('date')]).show()

+----+-----+----+------+----------+---------+---------+----------+-------+----+--------------------+------------+
|Year|Month|Hour|Minute|DayOfMonth|DayOfYear|DayOfWeek|WeekOfYear|Quarter|Beat|Location Description|Primary Type|
+----+-----+----+------+----------+---------+---------+----------+-------+----+--------------------+------------+
|   0|    0|   0|     0|         0|        0|        0|         0|      0|   0|                 213|           0|
+----+-----+----+------+----------+---------+---------+----------+-------+----+--------------------+------------+



The only column with N/A value in this case is Location Description. We can handle this by filling missing values or removing those rows. (In case Primary Type has N/A value, drop those rows as this is the target label). Lets look at how the data is distributed over the Location Description column

In [9]:
df.agg(F.countDistinct(F.col("Location Description")).alias("count")).show()

+-----+
|count|
+-----+
|  122|
+-----+



There are 122 distinct location descriptions. Lets take a look at the top 30 ones

In [10]:
df.groupBy("Location Description").count().sort(F.col("count").desc()).show(30)

+--------------------+------+
|Location Description| count|
+--------------------+------+
|              STREET|129033|
|            SIDEWALK| 88343|
|           RESIDENCE| 56639|
|           APARTMENT| 46895|
|               ALLEY| 15055|
|               OTHER| 14240|
|SCHOOL, PUBLIC, B...|  8945|
|RESIDENCE PORCH/H...|  8443|
|VEHICLE NON-COMME...|  7894|
|PARKING LOT/GARAG...|  7780|
|         GAS STATION|  6010|
|RESIDENTIAL YARD ...|  5378|
|     VACANT LOT/LAND|  5208|
|        CTA PLATFORM|  4836|
|POLICE FACILITY/V...|  4056|
|  GROCERY FOOD STORE|  3931|
|  SMALL RETAIL STORE|  3705|
|    RESIDENCE-GARAGE|  2847|
|CHA PARKING LOT/G...|  2837|
|          RESTAURANT|  2777|
|       PARK PROPERTY|  2732|
|CHA HALLWAY/STAIR...|  1968|
|SCHOOL, PUBLIC, G...|  1894|
|  ABANDONED BUILDING|  1805|
|       CHA APARTMENT|  1622|
|           CTA TRAIN|  1559|
|             CTA BUS|  1227|
|COMMERCIAL / BUSI...|  1075|
|   CONVENIENCE STORE|  1034|
| TAVERN/LIQUOR STORE|   985|
+---------

We can tell that most of the descriptions are among these top results. Since we only has 213 missing values in this case, lets just remove them. If we were to fill them, we could use the most frequent one (STREET), since it dominates the count above

In [11]:
df = df.dropna(subset=['Location Description']) #Subset not necessary in this case since only one column has N/A

Next step is to explore and filter the target column

In [12]:
#Distinct description types
df.agg(F.countDistinct(F.col("Primary Type")).alias("count")).show()

+-----+
|count|
+-----+
|   34|
+-----+



In [13]:
df.groupBy("Primary Type").count().sort(F.col("count").asc()).show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|        NON-CRIMINAL|    1|
|      NON - CRIMINAL|    2|
|           RITUALISM|    3|
|NON-CRIMINAL (SUB...|    3|
|OTHER NARCOTIC VI...|    4|
|   HUMAN TRAFFICKING|    6|
|    PUBLIC INDECENCY|    7|
|CONCEALED CARRY L...|   26|
|           OBSCENITY|   29|
|            STALKING|  175|
|        INTIMIDATION|  189|
|          KIDNAPPING|  309|
|LIQUOR LAW VIOLATION|  458|
|               ARSON|  688|
|            HOMICIDE| 1030|
|         SEX OFFENSE| 1038|
| CRIM SEXUAL ASSAULT| 1729|
|INTERFERENCE WITH...| 1856|
|            GAMBLING| 2072|
|OFFENSE INVOLVING...| 2487|
+--------------------+-----+
only showing top 20 rows



In [14]:
df.groupBy("Primary Type").count().sort(F.col("count").desc()).show(5)

+---------------+------+
|   Primary Type| count|
+---------------+------+
|      NARCOTICS|125473|
|        BATTERY| 90271|
|          THEFT| 51405|
|CRIMINAL DAMAGE| 37010|
|        ASSAULT| 27154|
+---------------+------+
only showing top 5 rows



The data is very unbalanced, and hence we should put the ones with very small values into OTHER CRIMES.

In [15]:
#List of target values below a threshold (e.g. 1000 counts)

rename_vals = df.groupBy("Primary Type").count().filter(
 (F.col("count") < 1000)).select("Primary Type").rdd.flatMap(lambda x: x).collect()

In [16]:
#User defined function which handles all rows below a threshold
@udf(StringType())
def balance_target(label):
    if label in rename_vals:
        return "OTHER CRIMES"
    else:
        return label

In [17]:
df = df.withColumn('y', balance_target(F.col('Primary Type'))).drop("Primary Type")

In [18]:
df.printSchema()

root
 |-- Day: date (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- DayOfMonth: integer (nullable = true)
 |-- DayOfYear: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- WeekOfYear: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Beat: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- y: string (nullable = true)



Year should be an integer, and Day should be removed

In [19]:
df = df.withColumn("_Year", df.Year.cast('integer')).drop(*["Day","Year"])

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

## Prepare the data for machine learning

### Index categorical data
The StringIndexer will encode a column of strings to a column of indices in the range [0,#labels).

In [41]:
categorical_cols = ["Beat", "Location Description"]

#Index feature columns
indexers = [ StringIndexer(inputCol=cat_col, outputCol="{}_idx".format(cat_col),
                           handleInvalid = 'error') for cat_col in categorical_cols] 

#Index target column too since it is categorical
target_indexer = [ StringIndexer(inputCol = 'y', outputCol = 'target', handleInvalid = 'error')]

### OneHotEncoding

Spark’s OneHotEncoder One-hot encoding maps a categorical feature, represented as a label index, to a binary vector with at most a single one-value indicating the presence of a specific feature value from among the set of all feature values. For string type input data, it is common to encode categorical features using StringIndexer first

In [22]:
encoders = [OneHotEncoder(dropLast=True,inputCol=idx.getOutputCol(), 
    outputCol="{}_catVec".format(idx.getOutputCol())) for idx in indexers]

### Vector assembler

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order.

We use this to define target and feature columns

In [31]:
feature_cols = ["_Year","Month","Hour","Minute","DayOfMonth",
                "DayOfYear","DayOfWeek","WeekOfYear","Quarter"] \
+ [enc.getOutputCol() for enc in encoders]

assembler = VectorAssembler(inputCols= feature_cols , outputCol="Features")

Now we are done with transformation steps, and can define a pipeline which executes the steps above.
(We could also do PCA, normalize, standardize etc.)

In [47]:
pipeline = Pipeline(stages = indexers + encoders + target_indexer  + [assembler])

pipeline_model = pipeline.fit(df)
final_df = pipeline_model.transform(df)

In [48]:
final_df.select(["Features","y","target"]).show()

+--------------------+-------------------+------+
|            Features|                  y|target|
+--------------------+-------------------+------+
|(162,[0,1,3,4,5,6...|CRIM SEXUAL ASSAULT|  18.0|
|(162,[0,1,3,4,5,6...|CRIM SEXUAL ASSAULT|  18.0|
|(162,[0,1,2,4,5,6...|        SEX OFFENSE|  19.0|
|(162,[0,1,2,4,5,6...| DECEPTIVE PRACTICE|  10.0|
|(162,[0,1,2,3,4,5...|          NARCOTICS|   0.0|
|(162,[0,1,4,5,6,7...| DECEPTIVE PRACTICE|  10.0|
|(162,[0,1,3,4,5,6...|        SEX OFFENSE|  19.0|
|(162,[0,1,2,3,4,5...|           BURGLARY|   8.0|
|(162,[0,1,2,4,5,6...|          NARCOTICS|   0.0|
|(162,[0,1,2,4,5,6...|              THEFT|   2.0|
|(162,[0,1,2,3,4,5...|  CRIMINAL TRESPASS|  11.0|
|(162,[0,1,2,3,4,5...| DECEPTIVE PRACTICE|  10.0|
|(162,[0,1,2,4,5,6...| DECEPTIVE PRACTICE|  10.0|
|(162,[0,1,3,4,5,6...|        SEX OFFENSE|  19.0|
|(162,[0,1,2,4,5,6...|          NARCOTICS|   0.0|
|(162,[0,1,2,3,4,5...|          NARCOTICS|   0.0|
|(162,[0,1,2,3,4,5...|          NARCOTICS|   0.0|


In [49]:
train, test = final_df.randomSplit([0.7, 0.3], seed = 42)

In [50]:
from pyspark.ml.classification import RandomForestClassifier

In [53]:
rf = RandomForestClassifier(featuresCol = 'Features', labelCol = 'target')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [55]:
predictions.select("_Year","Month","Hour","Minute","Location Description", 'y','target', 'prediction', 'probability').show()

+-----+-----+----+------+--------------------+--------------------+------+----------+--------------------+
|_Year|Month|Hour|Minute|Location Description|                   y|target|prediction|         probability|
+-----+-----+----+------+--------------------+--------------------+------+----------+--------------------+
| 2017|    1|   0|     0|           RESIDENCE|         SEX OFFENSE|  19.0|       1.0|[0.15440129281326...|
| 2012|    1|   0|     0|           RESIDENCE| CRIM SEXUAL ASSAULT|  18.0|       1.0|[0.14720829069423...|
| 2012|    1|   0|     0|           RESIDENCE|  DECEPTIVE PRACTICE|  10.0|       1.0|[0.14720829069423...|
| 2012|    1|   0|     0|           RESIDENCE|OFFENSE INVOLVING...|  14.0|       1.0|[0.14720829069423...|
| 2007|    1|   0|     0|           RESIDENCE| CRIM SEXUAL ASSAULT|  18.0|       1.0|[0.14720829069423...|
| 2007|    1|   0|     0|           RESIDENCE|               THEFT|   2.0|       1.0|[0.14837137862879...|
| 2001|    1|   0|     0|           R

In [56]:
predictions.groupBy("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|101731|
|       1.0| 33868|
|       2.0|    84|
+----------+------+



In [58]:
predictions.select(["target","prediction"]).show()

+------+----------+
|target|prediction|
+------+----------+
|  19.0|       1.0|
|  18.0|       1.0|
|  10.0|       1.0|
|  14.0|       1.0|
|  18.0|       1.0|
|   2.0|       1.0|
|  18.0|       1.0|
|  18.0|       1.0|
|   3.0|       1.0|
|  14.0|       1.0|
|   2.0|       1.0|
|  18.0|       1.0|
|   2.0|       1.0|
|  14.0|       1.0|
|   2.0|       1.0|
|  10.0|       1.0|
|  10.0|       0.0|
|  14.0|       1.0|
|  10.0|       0.0|
|  19.0|       1.0|
+------+----------+
only showing top 20 rows



In [64]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="target")
evaluator.evaluate(predictions)

0.22057741217411334

In [65]:
fi = rfModel.featureImportances.toArray()
fi

array([1.80129213e-02, 4.61913348e-03, 7.04044311e-02, 5.03294660e-02,
       1.88002037e-04, 1.29456020e-03, 1.55299801e-03, 2.27934311e-04,
       1.52769670e-03, 2.12061628e-02, 2.31569378e-05, 0.00000000e+00,
       9.54989725e-05, 6.03632108e-04, 1.10460641e-02, 8.20091796e-03,
       6.52470818e-05, 1.87598159e-03, 1.02189752e-03, 2.44306799e-04,
       5.04670934e-05, 1.19224780e-04, 9.64368130e-04, 1.02355873e-04,
       0.00000000e+00, 0.00000000e+00, 1.57703704e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.65301018e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.29458566e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.15239142e-02, 2.34971118e-01, 1.41233019e-01,
       2.05443448e-01, 1.03535858e-02, 1.61283590e-03, 4.45703081e-02,
       0.00000000e+00, 6.18648810e-03, 5.71929426e-03, 3.01252593e-03,
       0.00000000e+00, 0.00000000e+00, 3.29861186e-02, 2.01954780e-02,
      

We can see that a lot of features have no impact on the decisions, and hence we should reduce them

### Apply PCA 

Run a PCA algorithm on the vector assembler in order to reduce it. Lets see how the model behaves based on number of features

In [67]:
from pyspark.ml.feature import PCA

In [70]:
dims = [3,5,8,10,15,20,25,30]
for d in dims:
    
    pca = PCA(k=d, inputCol="Features", outputCol="PCA_Features")

    #Add to the ml pipeline
    pipeline = Pipeline(stages = indexers + encoders + target_indexer  + [assembler] + [pca])
    pipeline_model = pipeline.fit(df)
    final_df = pipeline_model.transform(df)

    #Split train and test
    train, test = final_df.randomSplit([0.7, 0.3], seed = 42)

    #Create a new random forest classifier based on the PCA features
    rf = RandomForestClassifier(featuresCol = 'PCA_Features', labelCol = 'target')
    rfModel = rf.fit(train)
    predictions = rfModel.transform(test)

    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="target")
    print("Accuracy with ",d, " features: ", evaluator.evaluate(predictions))

Accuracy with  3  features:  0.19956018704072448
Accuracy with  5  features:  0.2314908252937841
Accuracy with  8  features:  0.26832962006574956
Accuracy with  10  features:  0.26625662856928334
Accuracy with  15  features:  0.26166004856406316
Accuracy with  20  features:  0.25566383192355396
Accuracy with  25  features:  0.2528129498830853
Accuracy with  30  features:  0.25477064802534777


Visualize the distribution between test and prediction labels. We can see that it is unbalanced and overfitted since most of the target labels isnt predicted

In [88]:
pred_dist = predictions.groupBy("prediction").count()
org_dist = test.groupBy("target").count()

pred_dist.join(org_dist,pred_dist.prediction == org_dist.target,how="right").show()

+----------+-----+------+-----+
|prediction|count|target|count|
+----------+-----+------+-----+
|      null| null|   8.0| 4193|
|       0.0|94154|   0.0|37563|
|      null| null|   7.0| 5196|
|      null| null|  18.0|  531|
|       1.0|31083|   1.0|27254|
|      null| null|   4.0| 8156|
|      null| null|  11.0| 2426|
|      null| null|  14.0|  709|
|      null| null|   3.0|10941|
|      null| null|  19.0|  314|
|       2.0| 9708|   2.0|15365|
|      null| null|  17.0|  535|
|      10.0|  597|  10.0| 3071|
|      null| null|  13.0|  754|
|      null| null|   6.0| 5691|
|      null| null|  20.0|  324|
|      null| null|   5.0| 6174|
|      null| null|  15.0|  639|
|       9.0|  141|   9.0| 3268|
|      null| null|  16.0|  552|
+----------+-----+------+-----+
only showing top 20 rows



### Apply GridSearch and Kfold CV for model optimization

In [91]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

In [1]:
pca = PCA(k=8, inputCol="Features", outputCol="PCA_Features")

#Add to the ml pipeline
pipeline = Pipeline(stages = indexers + encoders + target_indexer  + [assembler] + [pca])
pipeline_model = pipeline.fit(df)
final_df = pipeline_model.transform(df)

#Split train and test
train, test = final_df.randomSplit([0.7, 0.3], seed = 42)

rf = RandomForestClassifier(featuresCol = 'PCA_Features', labelCol = 'target')

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 2, stop = 5, num = 2)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 3, stop = 7, num = 2)]) \
    .build()

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="target")

cv = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

cvModel = cv.fit(train)

predictions = cvModel.transform(test)

NameError: name 'PCA' is not defined

In [98]:
np.linspace(start = 5, stop = 25, num = 3)

array([ 5., 15., 25.])